In [1]:
# Importing necessary libraries from PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torch.nn.functional as F
import numpy as np

# Importing other necessary packages
import numpy as np
from PIL import Image

# Optional: for visualization and additional utilities
import matplotlib.pyplot as plt


# Load Pre-trained Model

In [2]:
from torchvision import models
import torch.nn as nn

# Load the pretrained MobileNetV3 model
model = models.mobilenet_v3_large(pretrained=True)

# Freeze all layers in the network
for param in model.parameters():
    param.requires_grad = False


/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Print the model architecture
print(model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

# Step 1: Load the data

In [4]:
from dataset_prep import *

In [5]:
# PARAMS
BATCH_SIZE = 200
NUM_WORKERS = 8
PIN_MEMORY = False
SHUFFLE = False # preprocessed data is already shuffled
LEARNING_RATE = 1e-3
NUM_EPOCHS = 100
PATIENCE = 5
# NUM_BATCHES = 10 this one when want to break early, train just one batch etc.
LOAD_MODEL = False

In [6]:
# Base path
base_path = '/Users/ruimaciel/Desktop/Barcelona/DL_Image/Final_project/data_prepped'

# Define the datasets with correct paths
train_ds = Prepped_Painting_Dataset(data_dir=os.path.join(base_path, 'train_prep'))
valid_ds = Prepped_Painting_Dataset(data_dir=os.path.join(base_path, 'val_prep'))
test_ds = Prepped_Painting_Dataset(data_dir=os.path.join(base_path, 'test_prep'))


train_loader = DataLoader(train_ds,
                          batch_size=BATCH_SIZE,
                          num_workers=NUM_WORKERS,
                          shuffle=SHUFFLE,
                          persistent_workers=True)


valid_loader = DataLoader(valid_ds,
                          batch_size=BATCH_SIZE,
                          num_workers=NUM_WORKERS,
                          shuffle=SHUFFLE,
                          persistent_workers=True)


test_loader = DataLoader(test_ds,
                          batch_size=BATCH_SIZE,
                          num_workers=NUM_WORKERS,
                          shuffle=SHUFFLE,
                          persistent_workers=True)

# Step 2: Unfreeze the Last Layer

In [7]:
# Assuming you have 'num_classes' as the number of your target classes
num_classes = 27  # Change this to your number of classes

# Replace the classifier - note for MobileNetV3, the final classifier is named 'classifier'
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

# Step 3: Move the Model to a GPU (Optional)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cpu


# Step 4: Define Loss Function and Optimizer

In [9]:
from torch.optim import Adam

criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = Adam(model.classifier.parameters(), lr=0.001, weight_decay=0.01)  # Add L2 regularization


# Step 5: Training the Model

In [28]:
# model.train()  # Set the model to training mode
# num_epochs = 5  # Number of training epochs

# # Lists to track losses
# train_losses = []
# val_losses = []

# for epoch in range(num_epochs):
#     # Training phase
#     model.train()  # Ensure the model is in training mode
#     running_train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device

#         # Convert labels to one-hot encoding
#         labels = torch.nn.functional.one_hot(labels.squeeze(), num_classes=26).float()

#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         # Backward and optimize
#         optimizer.zero_grad()  # Clear gradients w.r.t. parameters
#         loss.backward()  # Backward pass
#         optimizer.step()  # Perform a single optimization step

#         running_train_loss += loss.item()

#     epoch_train_loss = running_train_loss / len(train_loader)
#     train_losses.append(epoch_train_loss)

#     print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

#     # Validation phase
#     model.eval()  # Set the model to evaluation mode
#     running_val_loss = 0.0
#     with torch.no_grad():
#         for inputs, labels in valid_loader:
#             inputs, labels = inputs.to(device), labels.to(device)

#             # Convert labels to one-hot encoding
#             labels = torch.nn.functional.one_hot(labels.squeeze(), num_classes=26).float()

#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             running_val_loss += loss.item()

#     epoch_val_loss = running_val_loss / len(valid_loader)
#     val_losses.append(epoch_val_loss)

#     print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

# # Plotting training and validation loss
# plt.figure(figsize=(10, 5))
# plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
# plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()
# plt.show()


In [11]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [12]:
# Define the learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, min_lr=1e-6)

# Training and validation loops
num_epochs = 20  # Number of training epochs
patience = 2    # Number of epochs to wait for improvement
min_delta = 0.01  # Minimum change to qualify as an improvement

# Lists to track losses
train_losses = []
val_losses = []

# Early stopping variables
best_val_loss = float('inf')
early_stop_counter = 0
best_model_params = None

for epoch in range(num_epochs):
    # Training phase
    model.train()  # Ensure the model is in training mode
    running_train_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training")

    for inputs, labels in loop:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device

        # Ensure labels are 1D tensors of class indices
        if labels.ndimension() > 1:
            labels = labels.squeeze()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()  # Clear gradients w.r.t. parameters
        loss.backward()  # Backward pass
        optimizer.step()  # Perform a single optimization step

        running_train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    epoch_train_loss = running_train_loss / len(train_loader)
    train_losses.append(epoch_train_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Ensure labels are 1D tensors of class indices
            if labels.ndimension() > 1:
                labels = labels.squeeze()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

    epoch_val_loss = running_val_loss / len(valid_loader)
    val_losses.append(epoch_val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

    # Adjust the learning rate based on the validation loss
    scheduler.step(epoch_val_loss)

    # Early stopping check
    if epoch_val_loss < best_val_loss - min_delta:
        best_val_loss = epoch_val_loss
        early_stop_counter = 0
        best_model_params = model.state_dict().copy()  # Save the best model parameters
        print("Validation loss improved. Resetting early stopping counter and saving model parameters.")
    else:
        early_stop_counter += 1
        print(f"No improvement in validation loss. Early stopping counter: {early_stop_counter}")

    if early_stop_counter >= patience:
        print("Early stopping triggered. Stopping training.")
        break

# Load the best model parameters before plotting
if best_model_params is not None:
    model.load_state_dict(best_model_params)
    print("Loaded best model parameters from training.")

# Plotting training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()



Epoch [1/20] Training:   0%|          | 0/111 [00:00<?, ?it/s]

Epoch [1/20] Training:   0%|          | 0/111 [00:04<?, ?it/s]


KeyboardInterrupt: 

# Step 6: Evaluate the Model

In [31]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# Inference with Softmax and Metrics Calculation on Test Dataset
model.eval()  # Set the model to evaluation mode
softmax = nn.Softmax(dim=1)
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:  # Assume you have a DataLoader 'test_loader'
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probabilities = softmax(outputs)  # Apply softmax to get probabilities
        _, predicted = torch.max(probabilities, 1)  # Get the index of the max probability
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions) * 100
print(f'Overall Accuracy of the model on the test images: {accuracy:.2f}%')

# Calculate precision, recall, f1-score, and support for each class
precision, recall, f1, support = precision_recall_fscore_support(all_labels, all_predictions, average=None)

# Print precision, recall, f1-score, and accuracy for each class
for i in range(num_classes):
    class_precision = precision[i]
    class_recall = recall[i]
    class_f1 = f1[i]
    class_support = support[i]
    class_accuracy = 100 * (np.sum((np.array(all_labels) == i) & (np.array(all_predictions) == i))) / class_support
    print(f'Class {i}:')
    print(f'  Precision: {class_precision:.4f}')
    print(f'  Recall: {class_recall:.4f}')
    print(f'  F1 Score: {class_f1:.4f}')
    print(f'  Accuracy: {class_accuracy:.2f}%')

# Print overall precision, recall, f1-score
overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')
print(f'Overall Precision: {overall_precision:.4f}')
print(f'Overall Recall: {overall_recall:.4f}')
print(f'Overall F1 Score: {overall_f1:.4f}')

# Print classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=[f'Class {i}' for i in range(num_classes)]))



Overall Accuracy of the model on the test images: 39.84%
Class 0:
  Precision: 0.3682
  Recall: 0.3189
  F1 Score: 0.3418
  Accuracy: 22000.00%
Class 1:
  Precision: 0.2535
  Recall: 0.3280
  F1 Score: 0.2860
  Accuracy: 64700.00%
Class 2:
  Precision: 0.2123
  Recall: 0.3720
  F1 Score: 0.2703
  Accuracy: 87600.00%
Class 3:
  Precision: 0.4615
  Recall: 0.1295
  F1 Score: 0.2022
  Accuracy: 3900.00%
Class 4:
  Precision: 0.3969
  Recall: 0.3040
  F1 Score: 0.3443
  Accuracy: 38300.00%
Class 5:
  Precision: 0.5352
  Recall: 0.3500
  F1 Score: 0.4232
  Accuracy: 32700.00%
Class 6:
  Precision: 0.3333
  Recall: 0.0143
  F1 Score: 0.0274
  Accuracy: 300.00%
Class 7:
  Precision: 0.4692
  Recall: 0.5446
  F1 Score: 0.5041
  Accuracy: 13000.00%
Class 8:
  Precision: 0.5870
  Recall: 0.6260
  F1 Score: 0.6059
  Accuracy: 38500.00%
Class 9:
  Precision: 0.4282
  Recall: 0.2920
  F1 Score: 0.3472
  Accuracy: 34100.00%
Class 10:
  Precision: 0.3229
  Recall: 0.2280
  F1 Score: 0.2673
  Accuracy

## Another optimizer

# Unfreezing another layer

In [13]:

# Freeze all layers except the last two
for param in model.parameters():
    param.requires_grad = False

# Define the new classifier with dropout and the final linear layer
new_classifier = nn.Sequential(
    model.classifier[0],
    model.classifier[1],
    nn.Dropout(0.5),  # Add dropout before the final layer
    nn.Linear(model.classifier[0].out_features, num_classes)  # Add the final linear layer
)

model.classifier = new_classifier

# Unfreeze the last two layers
for param in model.classifier[-2:].parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.025, momentum=0.9, weight_decay=0.01)

cpu


In [14]:
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define the learning rate scheduler
scheduler = CosineAnnealingLR(optimizer, T_max=15, eta_min=1e-6)

# Training and validation loops
num_epochs = 20
patience = 5
min_delta = 0.01

train_accuracies = []
val_accuracies = []

best_val_accuracy = 0.0
early_stop_counter = 0
best_model_params = None

for epoch in range(num_epochs):
    model.train()
    running_corrects_train = 0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training")

    for inputs, labels in loop:
        inputs, labels = inputs.to(device), labels.to(device)
        if labels.ndimension() > 1:
            labels = labels.squeeze()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.classifier[-1].parameters(), max_norm=1.0)
        
        optimizer.step()
        running_corrects_train += torch.sum(preds == labels.data)
        loop.set_postfix(loss=loss.item())

    epoch_train_accuracy = running_corrects_train.double() / len(train_loader.dataset)
    train_accuracies.append(epoch_train_accuracy.item())
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {epoch_train_accuracy:.4f}')

    model.eval()
    running_corrects_val = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            if labels.ndimension() > 1:
                labels = labels.squeeze()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_corrects_val += torch.sum(preds == labels.data)

    epoch_val_accuracy = running_corrects_val.double() / len(valid_loader.dataset)
    val_accuracies.append(epoch_val_accuracy.item())
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {epoch_val_accuracy:.4f}')

    scheduler.step()

    if epoch_val_accuracy > best_val_accuracy + min_delta:
        best_val_accuracy = epoch_val_accuracy
        early_stop_counter = 0
        best_model_params = model.state_dict().copy()
        print("Validation accuracy improved. Resetting early stopping counter and saving model parameters.")
    else:
        early_stop_counter += 1
        print(f"No improvement in validation accuracy. Early stopping counter: {early_stop_counter}")

    if early_stop_counter >= patience:
        print("Early stopping triggered. Stopping training.")
        break

if best_model_params is not None:
    model.load_state_dict(best_model_params)
    print("Loaded best model parameters from training.")

plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()


Epoch [1/20] Training: 100%|██████████| 111/111 [13:49<00:00,  7.47s/it, loss=2.74]

Epoch [1/20], Training Accuracy: 0.1437


Epoch [1/20], Validation Accuracy: 0.3120
Validation accuracy improved. Resetting early stopping counter and saving model parameters.


Epoch [2/20] Training: 100%|██████████| 111/111 [15:26<00:00,  8.35s/it, loss=2.63]


Epoch [2/20], Training Accuracy: 0.2262
Epoch [2/20], Validation Accuracy: 0.3472
Validation accuracy improved. Resetting early stopping counter and saving model parameters.


Epoch [3/20] Training: 100%|██████████| 111/111 [15:51<00:00,  8.57s/it, loss=2.58]


Epoch [3/20], Training Accuracy: 0.2469
Epoch [3/20], Validation Accuracy: 0.3598
Validation accuracy improved. Resetting early stopping counter and saving model parameters.


Epoch [4/20] Training:   9%|▉         | 10/111 [01:27<14:39,  8.71s/it, loss=2.14]


KeyboardInterrupt: 

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support

# Define the label to art movement mapping
label_mapping = {0: 'Impressionism', 1: 'Expressionism', 2: 'Abstract_Expressionism', 3: 'Symbolism', 4: 'Color_Field_Painting', 
                 5: 'Ukiyo_e', 6: 'Post_Impressionism', 7: 'High_Renaissance', 8: 'Realism', 9: 'Pop_Art', 10: 'New_Realism', 
                 11: 'Minimalism', 12: 'Art_Nouveau_Modern', 13: 'Romanticism', 14: 'Fauvism', 15: 'Synthetic_Cubism', 
                 16: 'Baroque', 17: 'Cubism', 18: 'Analytical_Cubism', 19: 'Pointillism', 20: 'Northern_Renaissance', 
                 21: 'Mannerism_Late_Renaissance', 22: 'Contemporary_Realism', 23: 'Naive_Art_Primitivism', 
                 24: 'Early_Renaissance', 25: 'Rococo', 26: 'Unidentified'}

# Inference with Softmax and Metrics Calculation
model.eval()  # Set the model to evaluation mode
softmax = torch.nn.Softmax(dim=1)
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in valid_loader:  # Assume you have a DataLoader 'valid_loader'
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probabilities = softmax(outputs)  # Apply softmax to get probabilities
        _, predicted = torch.max(probabilities, 1)  # Get the index of the max probability
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions) * 100
print(f'Overall Accuracy of the model on the test images: {accuracy:.2f}%')

# Calculate precision, recall, f1-score, and support for each class
precision, recall, f1, support = precision_recall_fscore_support(all_labels, all_predictions, average=None)

# Print precision, recall, f1-score, and accuracy for each class
num_classes = len(label_mapping)
for i in range(num_classes):
    class_accuracy = 100 * (np.array(all_labels) == np.array(all_predictions)).sum() / support[i]
    print(f'Class {label_mapping[i]}:')
    print(f'  Precision: {precision[i]:.4f}')
    print(f'  Recall: {recall[i]:.4f}')
    print(f'  F1 Score: {f1[i]:.4f}')
    print(f'  Accuracy: {class_accuracy:.2f}%')

# Optionally, you can print the overall precision, recall, f1-score
overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')
print(f'Overall Precision: {overall_precision:.4f}')
print(f'Overall Recall: {overall_recall:.4f}')
print(f'Overall F1 Score: {overall_f1:.4f}')

# Create confusion matrix
cm = confusion_matrix(all_labels, all_predictions)

# Plot confusion matrix
plt.figure(figsize=(14, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[label_mapping[i] for i in range(num_classes)], 
            yticklabels=[label_mapping[i] for i in range(num_classes)])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()


In [ ]:
def saliency_map_batch(model, input_images, target_classes):
    model.eval()
    input_images.requires_grad_()
    outputs = model(input_images)
    loss = torch.nn.functional.cross_entropy(outputs, target_classes)
    model.zero_grad()
    loss.backward()

    saliency = input_images.grad.data.abs().max(dim=1)[0]
    saliency = saliency.cpu().numpy()
    
    return saliency

def visualize_saliency_maps(model, data_loader, num_images=5):
    images_shown = 0
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        saliency = saliency_map_batch(model, inputs, labels)
        
        for i in range(inputs.size(0)):
            if images_shown >= num_images:
                return
            
            plt.figure()
            plt.subplot(1, 2, 1)
            plt.imshow(inputs[i].cpu().permute(1, 2, 0))  # Assuming the input is in CHW format
            plt.axis('off')
            plt.subplot(1, 2, 2)
            plt.imshow(saliency[i], cmap='hot')
            plt.axis('off')
            plt.show()
            
            images_shown += 1

# Example usage:
visualize_saliency_maps(model, train_loader, num_images=5)



In [ ]:
def grad_cam(model, input_image, target_class):
    model.eval()
    input_image = input_image.unsqueeze(0).requires_grad_()
    output = model(input_image)
    loss = torch.nn.functional.cross_entropy(output, torch.tensor([target_class]).to(device))
    model.zero_grad()
    loss.backward()
    
    gradients = model.features[-1].weight.grad.data
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    
    activations = model.features(input_image).detach()
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    
    heatmap = torch.mean(activations, dim=1).squeeze()
    heatmap = torch.maximum(heatmap, torch.tensor(0.0))
    heatmap /= torch.max(heatmap)
    heatmap = heatmap.cpu().numpy()
    
    plt.matshow(heatmap)
    plt.show()
    
    heatmap = cv2.resize(heatmap, (input_image.shape[2], input_image.shape[3]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    superimposed_img = heatmap * 0.4 + input_image.cpu().numpy()
    plt.imshow(superimposed_img / 255)
    plt.show()

# Example usage:
input_image = train_loader.dataset[0][0].to(device)
target_class = train_loader.dataset[0][1]
grad_cam(model, input_image, target_class)

